for this PnL calculation we consider increase in balance / received coins as a purchase, and a decrease in balance / sent coins as a sell

In [1]:
import requests
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv('.env')
ckey = os.getenv('ckey')
auth = (ckey, '')

In [3]:
chainName = 'eth-mainnet'

walletAddress = '0x4CAECFc01ad2907d37F51d23e0691244f4B57692'

url = f"https://api.covalenthq.com/v1/{chainName}/address/{walletAddress}/portfolio_v2/"
        
headers = {
    "content-type": "application/json",
}
    
response = requests.get(url, headers=headers, auth=auth)

In [4]:
resp_json = response.json()

In [5]:
balances_df = pd.DataFrame()
prices_df = pd.DataFrame()

for coin_dict in resp_json['data']['items']:
    intermediate_balances_df = pd.DataFrame()
    intermediate_prices_df = pd.DataFrame()
    coin = coin_dict['contract_ticker_symbol']
    for day_dict in coin_dict['holdings']:
        index = day_dict['timestamp']
        try:
            price = float(day_dict['quote_rate'])
            num_coins = float(day_dict['close']['balance']) / 10**18
        except TypeError:
            price = 0
            num_coins = 0
        
        temp_balances_df = pd.DataFrame(data = {coin: num_coins}, index = [index])
        intermediate_balances_df = pd.concat([intermediate_balances_df, temp_balances_df])
        
        temp_prices_df = pd.DataFrame(data = {coin: price}, index = [index])
        intermediate_prices_df = pd.concat([intermediate_prices_df, temp_prices_df])
        
        
        
    balances_df[coin] = intermediate_balances_df[coin]
    prices_df[coin] = intermediate_prices_df[coin]

In [6]:
for df in [balances_df, prices_df]:
    for col_name in df.columns:  
        if (df[col_name] == 0).all():
            print(f'{col_name} is all 0s')
            for df2 in (balances_df, prices_df):
                del df2[col_name]

GRID is all 0s
DANK is all 0s
BEB is all 0s
uxtw.com$0.02回购 is all 0s
StableDAI.com is all 0s


In [7]:
prices_df = prices_df[::-1]
balances_df = balances_df[::-1]

### getting differences in balances

In [8]:
prices_df

,ETH,DAI,ENS,UNI,GTC,BANK
2023-02-03T00:00:00Z,1667.9390,1.005055,16.594004,7.118298,2.009736,0.008468
2023-02-04T00:00:00Z,1668.7157,1.002851,16.086983,7.212362,2.031683,0.008332
2023-02-05T00:00:00Z,1657.4211,1.014834,15.458372,6.978854,2.079368,0.008096
2023-02-06T00:00:00Z,1636.3623,1.004132,15.332073,6.795395,2.078488,0.008004
2023-02-07T00:00:00Z,1641.5417,0.981565,15.589381,6.817564,2.066487,0.006830
2023-02-08T00:00:00Z,1677.1360,1.030787,15.291176,6.829009,2.011186,0.007712
2023-02-09T00:00:00Z,1652.0455,1.011297,15.232785,6.747485,1.995974,0.007648
2023-02-10T00:00:00Z,1542.4635,1.001579,13.865210,6.378463,1.746544,0.007218
2023-02-11T00:00:00Z,1515.1930,0.996717,14.003132,6.376996,1.722840,0.007264
2023-02-12T00:00:00Z,1534.8944,1.001004,14.467163,6.533120,1.782675,0.007540


In [9]:
balances_df

,ETH,DAI,ENS,UNI,GTC,BANK
2023-02-03T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-04T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-05T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-06T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-07T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-08T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-09T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-10T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-11T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-12T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0


In [10]:
balances_diff_df = balances_df.diff().fillna(balances_df)
balances_diff_df

,ETH,DAI,ENS,UNI,GTC,BANK
2023-02-03T00:00:00Z,8.657813,10929.967886,735.269349,400.0,372.7633,10000.0
2023-02-04T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0
2023-02-05T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0
2023-02-06T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0
2023-02-07T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0
2023-02-08T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0
2023-02-09T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0
2023-02-10T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0
2023-02-11T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0
2023-02-12T00:00:00Z,0.000000,0.000000,0.000000,0.0,0.0000,0.0


In [11]:
trading_activity_df = balances_diff_df * prices_df
trading_activity_df

,ETH,DAI,ENS,UNI,GTC,BANK
2023-02-03T00:00:00Z,14440.703968,10985.216688,12201.062518,2847.3194,749.15601,84.68322
2023-02-04T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000
2023-02-05T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000
2023-02-06T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000
2023-02-07T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000
2023-02-08T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000
2023-02-09T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000
2023-02-10T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000
2023-02-11T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000
2023-02-12T00:00:00Z,0.000000,0.000000,0.000000,0.0000,0.00000,0.00000


In [12]:
principle_df = trading_activity_df.sum(axis=0)
principle_df

ETH     14440.703968
DAI     10985.216688
ENS     12201.062518
UNI      2847.319400
GTC       749.156010
BANK       84.683220
dtype: float64

In [13]:
final_balance_df = prices_df.tail(1) * balances_df.tail(1)
final_balance_df

,ETH,DAI,ENS,UNI,GTC,BANK
2023-03-05T00:00:00Z,13583.789134,10951.957888,10310.862957,2466.38068,659.06747,68.47141


In [14]:
roi_df = final_balance_df/principle_df
roi_df

,ETH,DAI,ENS,UNI,GTC,BANK
2023-03-05T00:00:00Z,0.94066,0.996972,0.845079,0.866211,0.879747,0.808559


In [15]:
roi_df = final_balance_df/principle_df
roi_df

,ETH,DAI,ENS,UNI,GTC,BANK
2023-03-05T00:00:00Z,0.94066,0.996972,0.845079,0.866211,0.879747,0.808559


In [16]:
print(f"Best ROI:\t{roi_df.idxmax(axis=1)[0]}, {(roi_df.max(axis=1)[0]-1)*100}%")
print(f"Worst ROI:\t{roi_df.idxmin(axis=1)[0]}, {(roi_df.min(axis=1)[0]-1)*100}%")

Best ROI:	DAI, -0.30275961071971613%
Worst ROI:	BANK, -19.144064195952872%


In [17]:
pnl_df = final_balance_df-principle_df
pnl_df

,ETH,DAI,ENS,UNI,GTC,BANK
2023-03-05T00:00:00Z,-856.914834,-33.258799,-1890.199561,-380.93872,-90.08854,-16.21181


In [18]:
print(f"Best PnL:\t{pnl_df.idxmax(axis=1)[0]}, ${pnl_df.max(axis=1)[0]}")
print(f"Worst PnL:\t{pnl_df.idxmin(axis=1)[0]}, ${pnl_df.min(axis=1)[0]}")

Best PnL:	BANK, $-16.21181
Worst PnL:	ENS, $-1890.199561088717
